In [2]:
import pandas as pd
import os
from src.surface import *
from src.view_profile import *
from src.building import Building
from src.site import *

from pathlib import Path

In [36]:
class TestCase:
    def __init__(self, name, latitude=-27.48637226, longitude=153.01796166,
                 tz='Australia/Brisbane', azimuth=0,
                 surface_path='src/torbreck/surface_definitions.csv',
                 apartments_per_level=7, levels=13, solar_ci=0.048,
                 battery_capacity=0
                 ):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.tz = tz
        self.azimuth = azimuth
        self.battery_capacity = battery_capacity
        self.solar_ci = solar_ci

        self.surface_path = surface_path
        self.apartments_per_level = apartments_per_level
        self.levels = levels

    def run(self):
        self.site = Site(longitude=self.longitude,
                         latitude=self.latitude, tz=self.tz)

        self.building = Building(
            self.name, self.azimuth, self.site, self.levels)
        self.building.load_surfaces_from_csv(self.surface_path)
        self.building.calculate_irradiance()

        self.building.generated_power

    def calculate_results(self):
        grid_ci_df = pd.read_csv(
            'data/processed/grid_ci.csv', index_col='hour')
        demand_ts_df = pd.read_csv(
            'data/processed/demand_timeseries.csv', index_col='Datetime')
        demand_ts_df.index = pd.to_datetime(demand_ts_df.index)
        demand_ts_df.index = demand_ts_df.index.tz_localize(self.tz)

        building_ts = demand_ts_df * self.apartments_per_level * self.levels

        # Create a column with the hour of day for joining
        dummy = building_ts.copy()
        dummy['hour'] = dummy.index.hour

        # Merge the emissions intensity into the demand dataframe
        dummy = dummy.merge(
            grid_ci_df,
            left_on='hour',
            right_index=True,
            how='left'
        )

        # Compute emissions in kg CO₂e
        building_ts['Grid Emissions (kg CO2e/kWh)'] = dummy['Emissions Intensity - kgCO₂e/KWh']
        building_ts['Generation (kWh)'] = self.building.generated_power.groupby('Datetime')[
            'Generation (kWh)'].sum()

        building_ts['Solar Excess (kWh)'] = 0.0
        building_ts['Solar Supply (kWh)'] = 0.0
        building_ts['Grid Supply (kWh)'] = 0.0
        building_ts['Battery Charge Level (kWh)'] = 0.0
        building_ts['Battery Charge/Discharge (kWh)'] = 0.0
        building_ts['Irradiance (W/m2)'] = self.site.tmy.copy().sum(axis=1)
        building_ts['Hour of Day'] = building_ts.index.hour

        # Do it
        previous_ts_battery_level = 0
        for dt, row in building_ts.iterrows():
            net_supply = row['Generation (kWh)'] - row['Demand (KWh)']
            charge_diff = 0
            # If surplus energy being generated store if possible
            if net_supply > 0:
                charge_diff = min(net_supply, self.battery_capacity -
                                  previous_ts_battery_level)
                net_supply = net_supply - charge_diff
            # If surplus -ve and battery level is greater than 0 discharge available power
            elif previous_ts_battery_level > 0:
                charge_diff = max(-previous_ts_battery_level, net_supply)
                net_supply -= charge_diff

            building_ts.loc[dt, 'Solar Supply (kWh)'] = min(
                row['Generation (kWh)'], row['Demand (KWh)'])
            building_ts.loc[dt, 'Grid Supply (kWh)'] = -min(0, net_supply)
            building_ts.loc[dt, 'Solar Excess (kWh)'] = max(0, net_supply)
            building_ts.loc[dt, 'Battery Charge/Discharge (kWh)'] = charge_diff
            building_ts.loc[dt,
                            'Battery Charge Level (kWh)'] = previous_ts_battery_level + charge_diff

            previous_ts_battery_level += charge_diff

        building_ts['Emissions (kg CO2e)'] = building_ts['Grid Supply (kWh)'] * building_ts['Grid Emissions (kg CO2e/kWh)'] + (
            building_ts['Solar Supply (kWh)'] - building_ts['Battery Charge/Discharge (kWh)'].clip(upper=0)) * self.solar_ci

        building_ts['Battery Discharge'] = - \
            building_ts['Battery Charge/Discharge (kWh)'].clip(upper=0)
        building_ts['Battery Charge'] = building_ts['Battery Charge/Discharge (kWh)'].clip(
            lower=0)
        building_ts['Solar Share of Demand'] = building_ts['Solar Supply (kWh)'] / \
            building_ts['Demand (KWh)']
        building_ts['Solar + Battery Share of Demand'] = (building_ts['Solar Supply (kWh)'] -
                                                          building_ts['Battery Charge/Discharge (kWh)'].clip(upper=0)) / building_ts['Demand (KWh)']

        self.results = building_ts
        return building_ts

In [ ]:
tests = [
    TestCase('Full retrofit'),
    TestCase('Facade retrofit',
             surface_path='src/torbreck/facade_definitions.csv'),
    TestCase('Balcony retrofit',
             surface_path='src/torbreck/balcony_definitions.csv'),
    TestCase('Retrofit with 180 kWh Battery', battery_capacity=180),
    TestCase('Retrofit facing East', azimuth=90),
    TestCase('Retrofit in Hobart', longitude=147.3257, latitude=-42.8826),
    TestCase('Retrofit in Cairns', longitude=145.7710, latitude=-16.9203),
]

In [55]:
for test in tests:
    test.run()
    results_df = test.calculate_results()

    Path(f"results/test_cases/{test.name}").mkdir(parents=True, exist_ok=True)
    test.building.generated_power.to_csv(
        f"results/test_cases/{test.name}/generation_results.csv")
    results_df.to_csv(f"results/test_cases/{test.name}/analysis_results.csv")

/workspace/src/site.py:45: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  tmy.index = tmy.index + pd.DateOffset(year=2013)
/workspace/src/building.py:77: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_long = captured.stack(level=[0, 1]).reset_index()


### Collates results from all test cases

In [56]:
summary = pd.DataFrame(
    columns=('Test', 'Carbon Emissions (kg CO2e)', 'Solar to Grid (kWh)', 'Grid Demand (kWh)', 'Solar Supply (kWh)', 'Electricity CI (kg CO2e/kWh)', 'Battery Supply'))
summary = summary.set_index('Test')


for test_name in os.listdir('results/test_cases'):
    test_results = pd.read_csv(
        f'results/test_cases/{test_name}/analysis_results.csv')

    if 'Pre Retrofit'not in test_results.index:
        summary.loc['Pre Retrofit'] = {
            'Carbon Emissions (kg CO2e)': (test_results['Grid Emissions (kg CO2e/kWh)'] * test_results['Demand (KWh)']).sum(),
            'Solar to Grid (kWh)': 0,
            'Grid Demand (kWh)': test_results['Demand (KWh)'].sum(),
            'Solar Supply (kWh)': 0,
            'Battery Supply': 0,
            'Electricity CI (kg CO2e/kWh)': (test_results['Grid Emissions (kg CO2e/kWh)'] * test_results['Demand (KWh)']).sum() / test_results['Demand (KWh)'].sum()
        }
    # Average CI Intensity
    total_carbon_emissions = test_results['Emissions (kg CO2e)']
    total_grid_demand = test_results['Grid Supply (kWh)']
    total_solar_supply = test_results['Solar Supply (kWh)']
    total_solar_to_grid = test_results['Solar Excess (kWh)']

    summary.loc[test_name] = {
        'Carbon Emissions (kg CO2e)': total_carbon_emissions.sum(),
        'Solar to Grid (kWh)': total_solar_to_grid.sum(),
        'Grid Demand (kWh)': total_grid_demand.sum(),
        'Solar Supply (kWh)': total_solar_supply.sum(),
        'Electricity CI (kg CO2e/kWh)': total_carbon_emissions.sum()/test_results['Demand (KWh)'].sum(),
        'Battery Supply': test_results['Battery Discharge'].sum()
    }

summary.to_csv('results/summary.csv')

### Combine generation and analysis data

In [ ]:
analysis_all = {}
generation_all = {}
for test_name in os.listdir('results/test_cases'):
    analysis_all[test_name] = pd.read_csv(
        f'results/test_cases/{test_name}/analysis_results.csv')

    if len(analysis_all) == 1:
        pre = analysis_all[test_name].copy()
        pre['Grid Supply (kWh)'] = pre['Demand (KWh)']
        pre[['Generation (kWh)', 'Solar Excess (kWh)',
             'Solar Supply (kWh)']] = 0
        pre['Emissions (kg CO2e)'] = pre['Grid Supply (kWh)'] * \
            pre['Grid Emissions (kg CO2e/kWh)']
        analysis_all['Pre Retrofit'] = pre

    gen = pd.read_csv(f'results/test_cases/{test_name}/generation_results.csv')
    gen['Hour of the Day'] = pd.to_datetime(gen['Datetime']).dt.hour
    generation_all[test_name] = gen


combined_analysis = pd.concat(analysis_all).reset_index(
    level=0).rename(columns={'level_0': 'Test name'})
combined_generation = pd.concat(generation_all).reset_index(level=0).rename(
    columns={'level_0': 'Test name'}).drop(columns=['Unnamed: 0'])